# Hello PyOpenCL

In [1]:
import pyopencl as cl
import numpy as np
import numpy.linalg as la

mf = cl.mem_flags

This notebook demonstrates the simplest PyOpenCL workflow that touches all essential pieces:

* Data transfer
* Kernel compilation
* Execution

In [2]:
a = np.random.rand(50000).astype(np.float32)

Now create a context.

In [5]:
ctx = cl.create_some_context(interactive=True)

queue = cl.CommandQueue(ctx)

Choose platform:
[0] <pyopencl.Platform 'Intel(R) OpenCL' at 0x2e671d8>


Choice [0]: 


Set the environment variable PYOPENCL_CTX='' to avoid being asked again.


Now allocate a buffer. `Buffer(context, flags, size=None, hostbuf=None)`

In [6]:
a_buf = cl.Buffer(ctx, mf.READ_WRITE, size=a.nbytes)

Then transfer data:

In [7]:
cl.enqueue_copy(queue, a_buf, a)

Here's our kernel source code:

In [8]:
prg = cl.Program(ctx, """
    __kernel void twice(__global float *a)
    {
      int gid = get_global_id(0);
      a[gid] = 2*a[gid];
    }
    """).build()

/home/andreas/src/pyopencl/pyopencl/__init__.py:250: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


Run the kernel.

In [9]:
prg.twice(queue, a.shape, None, a_buf)

Copy the data back.

In [10]:
result = np.empty_like(a)

cl.enqueue_copy(queue, result, a_buf)

Check the result.

In [11]:
print(la.norm(result - 2*a), la.norm(a))

0.0 129.52843
